### Convolutional Neural Network

<img src="https://ujwlkarn.files.wordpress.com/2016/07/convolution_schematic.gif?w=268&h=196&zoom=2"/>

<img src="../CNN_Static.png"/>

In [1]:
# Formula for above Convoluted Feature 

# input  - 5X5 
# filter - 3X3
# stride - 1X1

# output = (input - filter + stride)/stride (5-3+1)/(1)=3 

In [3]:
# (6 - 3 + 2)/2 = 5/2 = 2.5 = ~3  
# (6 - 4 + 2)/2 = 2

In [4]:
#1. Filter 
#2. Stride 
#3. Padding (if required)
#4. Depth (how many filters)
#5. 

In [ ]:
#1. CNN 
#2. Pooling 

#3. CNN
#4. Pooling 

#5. CNN
#6. Pooling 

#7. Flatten
#8. Dense 
#9. Dense 


# CNN + Pooling = Chunk

In [5]:
# Generally 
# if image size is less than 500 * 500 
# CNN 
# Filter 3*3 
# Stride 1*1
# Filter number - ?? 

# Pooling 
# Filter 2*2 
# Stride 2*2 


# How many chunk ?? 
# Every chuck has a responsibility 
# Early Convolution they work on edges 
# Shapes 
# Work on detailed structure 

In [1]:
! pip uninstall -y kaggle
! pip install -q --upgrade  kaggle
! pip freeze | grep kaggle 

Uninstalling kaggle-1.5.10:
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 61kB 4.3MB/s 
kaggle==1.5.10


In [2]:
! mkdir /root/.kaggle
! mv kaggle.json /root/.kaggle

In [3]:
import kaggle

In [4]:
!kaggle competitions download -c dog-breed-identification

 99% 686M/691M [00:03<00:00, 260MB/s]
100% 691M/691M [00:03<00:00, 195MB/s]


In [5]:
! unzip -q dog-breed-identification.zip

In [6]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf

In [7]:
label = pd.read_csv("labels.csv")

In [8]:
label.head()

id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever

In [9]:
label["filename"] = label.id + ".jpg"

In [10]:
label.head()

id  ...                              filename
0  000bec180eb18c7604dcecc8fe0dba07  ...  000bec180eb18c7604dcecc8fe0dba07.jpg
1  001513dfcb2ffafc82cccf4d8bbaba97  ...  001513dfcb2ffafc82cccf4d8bbaba97.jpg
2  001cdf01b096e06d78e9e5112d419397  ...  001cdf01b096e06d78e9e5112d419397.jpg
3  00214f311d5d2247d5dfe4fe24b2303d  ...  00214f311d5d2247d5dfe4fe24b2303d.jpg
4  0021f9ceb3235effd7fcde7f7538ed62  ...  0021f9ceb3235effd7fcde7f7538ed62.jpg

[5 rows x 3 columns]

In [11]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.2, rotation_range=30, horizontal_flip=True,rescale=1/255, validation_split=0.2)

In [12]:
train_idg = idg.flow_from_dataframe(label, "train", x_col="filename", y_col="breed", target_size=(200, 200), batch_size=64, subset="training")
val_idg   = idg.flow_from_dataframe(label, "train", x_col="filename", y_col="breed", target_size=(200, 200), batch_size=64, subset="validation")

Found 8178 validated image filenames belonging to 120 classes.
Found 2044 validated image filenames belonging to 120 classes.


In [16]:
# del model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, kernel_size=3, strides=(1,1), activation=tf.keras.activations.relu, input_shape=(200, 200, 3), name="1stConv"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=None, name="1stMaxPool"))

model.add(tf.keras.layers.Conv2D(32, kernel_size=3, strides=(1,1), activation=tf.keras.activations.relu, name="2ndConv"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=None, name="2ndMaxPool"))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation=tf.keras.activations.relu, name="1stDense"))
model.add(tf.keras.layers.Dense(120, activation=tf.keras.activations.softmax, name="2ndDense"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1stConv (Conv2D)             (None, 198, 198, 16)      448       
_________________________________________________________________
1stMaxPool (MaxPooling2D)    (None, 99, 99, 16)        0         
_________________________________________________________________
2ndConv (Conv2D)             (None, 97, 97, 32)        4640      
_________________________________________________________________
2ndMaxPool (MaxPooling2D)    (None, 48, 48, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 73728)             0         
_________________________________________________________________
1stDense (Dense)             (None, 256)               18874624  
_________________________________________________________________
2ndDense (Dense)             (None, 120)              

In [17]:
model.compile(tf.keras.optimizers.SGD(), tf.keras.losses.categorical_crossentropy, metrics=["acc"])

In [ ]:
model_history = model.fit(train_idg, epochs=10, validation_data=val_idg)

Epoch 1/10
128/128 [==============================] - 308s 2s/step - loss: 8.9022 - acc: 0.0065 - val_loss: 8.0698 - val_acc: 0.0073
Epoch 2/10
128/128 [==============================] - 303s 2s/step - loss: 7.8461 - acc: 0.0044 - val_loss: 7.6762 - val_acc: 0.0117
Epoch 3/10
128/128 [==============================] - 305s 2s/step - loss: 7.3165 - acc: 0.0064 - val_loss: 7.1130 - val_acc: 0.0117
Epoch 4/10
128/128 [==============================] - 302s 2s/step - loss: 6.9621 - acc: 0.0066 - val_loss: 7.0922 - val_acc: 0.0117
Epoch 5/10
128/128 [==============================] - 305s 2s/step - loss: 6.8744 - acc: 0.0066 - val_loss: 7.0754 - val_acc: 0.0117
Epoch 6/10
128/128 [==============================] - 302s 2s/step - loss: 6.9358 - acc: 0.0069 - val_loss: 6.8038 - val_acc: 0.0117
Epoch 7/10
128/128 [==============================] - 305s 2s/step - loss: 6.7337 - acc: 0.0063 - val_loss: 6.7947 - val_acc: 0.0117
Epoch 8/10
128/128 [==============================] - 303s 2s/step - 